In [15]:
paths_to_results = [
    "../../../results/temp_baselines/lcb_starcoder2_3b_baseline_temp08.json.gz",
    "../../../results/temp_finetunes/lcb_starcoder2_3b_taco_single_temp08.json.gz",
    "../../../results/temp_finetunes/lcb_starcoder2_3b_taco_full_temp08.json.gz",
]

In [16]:
from pathlib import Path
for p in paths_to_results:
    assert Path(p).exists(), f"Path {p} doesn't exist!"

In [ ]:
from codeprm.eval.metrics import get_pass_ks
from codeprm.eval.metrics import get_pass_ks

pass_ks = {}
upper_k = 
for r in results: